# Imports

In [290]:
import pandas as pd
import os 
current_folder = os.getcwd()
print(current_folder)
parent_folder = os.path.dirname(current_folder)
os.chdir(parent_folder)

/home/araceli/code/energy-analysis-and-predictions


In [333]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

from utils.parameters import ENERGY_COMPANIES_FOLDER, ENERGY_COMPANY_SECTOR,\
                            DICT_COMPANIES_NAME_TO_CHANGE, \
                            LIST_WEIRD_FIRST_RAW

from load_data.load_energy_produced_by_companies import load_company_data_by_sector

from predict_data.create_train_test_data import get_scaled_X_y_train_and_test_data
from predict_data.create_simple_train_test_data import get_X_y_scaled_and_splited                       
from predict_data.load_gru_model import load_gru_model          
from predict_data.model_methods import compile_model, train_model,evaluate_model, predict_model                                          

from visualize_data.plot_price_with_interactive_timestamps import plot_price_with_interactive_timestamps   
from visualize_data.plot_model_history import plot_history
from visualize_data.plot_predict_vs_test import plot_predict_vs_test

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Company Sectors

In [328]:
energy_sectors = pd.read_csv(ENERGY_COMPANY_SECTOR , delimiter = ",", encoding = "utf-8") 
energy_sectors['Energy_Source'] = energy_sectors['Energy_Source'].map(lambda x: str(x)[:-1])
energy_sectors['Name'] = energy_sectors['Name'].map(lambda x: str(x)[:-1])
energy_sectors['Name'] = energy_sectors['Name'].map(lambda x: str(x).replace(' ', '-'))
energy_sectors['Name'] = energy_sectors['Name'].map(lambda x: str(x).replace('_', '-'))
energy_sectors = energy_sectors.dropna()
energy_sectors = energy_sectors.drop_duplicates()
energy_sectors.head()

,Name,Place,Energy_Source,Rated_capacity
0,Abfallentsorgungszentrum-Asdonkshof,Kamp-Lintfort,Abfall,20.8 MW
1,Abfallheizkraftwerk-Neunkirchen,Neunkirchen,Abfall,11.6 MW
2,AMK---Abfallentsorgungsgesellschaft-des-Maerki...,Iserlohn,Abfall,12.6 MW
3,AVA-GmbH,Augsburg,Abfall,10 MW
4,AVA-Velsen,Saarbruecken,Abfall,16 MW


In [329]:
cond = energy_sectors['Name'].str.contains('Ludwigshafen') 
#cond = energy_sectors['Energy_Source'] == 'Laufwasser'
#cond = energy_sectors['Name'] == 'Boxberg'
#test = energy_sectors[cond].sort_values(by='Name')
#test['Name'][532]
test = energy_sectors[cond]
#test['Name'].values
test

,Name,Place,Energy_Source,Rated_capacity
16,Fernheizkraftwerk-Ludwigshafen,Ludwigshafen,Abfall,28 MW
160,Blockheizkraftwerk-Ludwigshafen,Ludwigshafen,Erdgas,"12,5 MW"
680,Kraftwerk-BASF-Ludwigshafen-Nord,Ludwigshafen,Sonstige konventionelle Energietraeger,85 MW
681,Kraftwerk-BASF-Ludwigshafen-Mitte,Ludwigshafen,Erdgas,"1.074,5 MW"
684,Kraftwerk-BASF-Ludwigshafen-Sued,Ludwigshafen,Erdgas,410 MW


In [284]:
cond2 = energy_sectors['Energy_Source']=='Wind (Onshore)' 
#cond = energy_sectors['Energy_Source'] == 'Laufwasser'
#cond = energy_sectors['Name'] == 'Boxberg'
#test = energy_sectors[cond].sort_values(by='Name')
#test['Name'][532]
test2 = energy_sectors[cond2]
test2.reset_index(drop=True, inplace=True)
test2[80:]

,Name,Place,Energy_Source,Rated_capacity
80,Windpark-Weimar,Bucha,Wind (Onshore),12 MW
81,Windpark-Weilrod,Weilrod,Wind (Onshore),"16,8 MW"
82,Windpark-Weener-Verwaltungs-GmbH,Weener,Wind (Onshore),"21,5 MW"
83,Windpark-Wangelau,Wangelau,Wind (Onshore),"14,4 MW"
84,Windpark-Wanderup-West-GmbH-&-Co.-KG,Wanderup,Wind (Onshore),"15,9 MW"
85,Windpark-Wanderup,Wanderup,Wind (Onshore),"34,5 MW"
86,Windpark-Walsleben-Goldbeck,Goldbeck,Wind (Onshore),"18,3 MW"
87,Windpark-Walsleben-Goldbeck,Hansestadt Osterburg,Wind (Onshore),"21,3 MW"
88,Windpark-Waldow,Waldow,Wind (Onshore),28 MW


In [285]:
DICT_COMPANIES_NAME_TO_CHANGE = {'Altenw_rth': 'Altenwoerth',
                       'H_usling':'Haeusling',
                       'S_ckingen':'Saeckingen',
                       'Ro_hag':'Rosshag',
                       'M_nchen_Nord_2':'Muenchen-Nord-2',
                       'Linz-S_d_Fernheizkraftwerk':'Linz-Sued-Fernheizkraftwerk',
                       'Gro_kraftwerk_Mannheim': 'Grosskraftwerk-Mannheim',
                       'K_stenkraftwerk_K.I.E.L.':'Kuestenkraftwerk K.I.E.L', 
                       'Heizkraftwerk_R_merbr_cke': 'Heizkraftwerk-Roemerbruecke'}

In [69]:
text='Heizkraftwerk_R_merbr_cke'
if text in DICT_COMPANIES_NAME_TO_CHANGE.keys():
    text=DICT_COMPANIES_NAME_TO_CHANGE[text]
print(text)

Heizkraftwerk-Roemerbruecke


In [56]:
energy_sector_types = energy_sectors['Energy_Source'].value_counts()
energy_sector_types

Erdgas                                    269
Laufwasser                                104
Wind (Onshore)                             90
Steinkohle                                 80
Photovoltaik                               63
Biomasse                                   56
na                                         55
Abfall                                     45
Braunkohle                                 28
Pumpspeicher                               28
Wind (Offshore)                            28
Sonstige konventionelle Energietraeger     25
Mineraloelprodukte                         23
Speicherwasser (ohne Pumpspeicher)         21
Batteriespeicher                           15
Kernenergie                                 8
Steinkohle - Heizoe                         1
Biomass                                     1
Waerme                                      1
Erdgas – Heizoe                             1
Name: Energy_Source, dtype: int64

In [21]:
cond = energy_sectors['Name'].str.contains('Abwinden')
#cond = energy_sectors['Name'] == ' Abwinden-Asten'
energy_sectors[cond]

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [64]:
dict_companies = {sector:[] for sector in sectors}
dict_companies

{'Abfall': [],
 'Batteriespeicher': [],
 'Biomass': [],
 'Biomasse': [],
 'Braunkohle': [],
 'Erdgas': [],
 'Erdgas – Heizoe': [],
 'Kernenergie': [],
 'Laufwasser': [],
 'Mineraloelprodukte': [],
 'Photovoltaik': [],
 'Pumpspeicher': [],
 'Sonstige konventionelle Energietraeger': [],
 'Speicherwasser (ohne Pumpspeicher)': [],
 'Steinkohle': [],
 'Steinkohle - Heizoe': [],
 'Waerme': [],
 'Wind (Offshore)': [],
 'Wind (Onshore)': [],
 'na': []}

# Power Plant data

In [83]:
#foldername = os.path.join(ENERGY_COMPANIES_FOLDER,'power-plant-data-2022-Jan-to-Dec' )
power_plant_list = os.listdir(ENERGY_COMPANIES_FOLDER)
power_plant_list.sort()
print('len(power_plant_list)=', len(power_plant_list))


len(power_plant_list)= 224


In [86]:
for file in power_plant_list:
    if 'Identifier' in file:
        power_plant_list.remove(file)
        
len(power_plant_list)

114

## Change delimiter

In [313]:
name ='Kraftwerk_BASF_Ludwigshafen_Mitte'
index=None
for file in power_plant_list:
    if name in file:
        index = power_plant_list.index(file)       
        print(index)

58


In [317]:
filename = os.path.join(ENERGY_COMPANIES_FOLDER, power_plant_list[index])
print(filename)
df = pd.read_csv(filename, delimiter=';')
#df.to_csv(filename, sep=';', index=False)
df.head()

/home/araceli/code/energy-analysis-and-predictions/raw_data/energy-companies/power-plant-data-2022-Jan-to-Dec/Kraftwerk_BASF_Ludwigshafen_Mitte_202201010000_202212312359_Tag_95.csv


,Datum;Anfang;Ende;Generation_DE [MW] Berechnete Auflösungen;Generation_DE GUD A 800
0,DT 10 [MW] Berechnete Auflösungen;Generation_...
1,DT 10 [MW] Berechnete Auflösungen
2,NaN
3,NaN
4,NaN


In [303]:
df = pd.read_csv(filename, delimiter=';', decimal=',')
df['Generation_DE Koepchenwerk [MW] Berechnete Auflösungen'].isna().value_counts()/df.shape[0]

False    1.0
Name: Generation_DE Koepchenwerk [MW] Berechnete Auflösungen, dtype: float64

In [306]:
type(df['Generation_DE Koepchenwerk [MW] Berechnete Auflösungen'][0])

str

In [156]:
if '(3 GT + 1 DT, Sammelschiene)' in df['Datum'][0]:
    df= df.iloc[1:,:]
df.head()

,Datum,Anfang,Ende,Generation_DE Heizkraftwerk Dresden-Nossener Brücke
1,01.01.2022,00:00,00:00,1.284
2,02.01.2022,00:00,00:00,1.354
3,03.01.2022,00:00,00:00,2.328
4,04.01.2022,00:00,00:00,4.449
5,05.01.2022,00:00,00:00,3.913


In [259]:
if type(df['Datum'][0])==str:
    df= df.iloc[1:,:]
    df.reset_index(drop=True, inplace=True)
df.head()

,Datum,Anfang,Ende,Generation_DE Heizkraftwerk Dresden-Nossener Brücke
0,01.01.2022,00:00,00:00,1.284
1,02.01.2022,00:00,00:00,1.354
2,03.01.2022,00:00,00:00,2.328
3,04.01.2022,00:00,00:00,4.449
4,05.01.2022,00:00,00:00,3.913


In [256]:
#type(df['Datum'][0])
df.columns

Index(['Datum', 'Anfang', 'Ende',
       'Generation_DE Heizkraftwerk Dresden-Nossener Brücke '],
      dtype='object')

In [261]:
def clean_column_dataframe(text):
    text =str(text)
    text=text.replace('.', '')
    text=text.replace(',', '.')
    text=text.replace('-', np.nan)
    text=text.astype('float32')
    text= text/1000 # true scale
    return text

In [262]:
df['new']=df['Generation_DE Heizkraftwerk Dresden-Nossener Brücke '].apply(clean_column_dataframe)
df

TypeError: replace() argument 2 must be str, not float

In [114]:
cols = list(df.columns)
for col in cols:
    print(col)
    if 'Unnamed' in col:
        df=df.drop(columns=col)
df.head()
    

Datum
Anfang
Ende
Generation_DE Block 4 [MW] Berechnete Auflösungen
Unnamed: 4


,Datum,Anfang,Ende,Generation_DE Block 4 [MW] Berechnete Auflösungen
0,01.01.2022,00:00,00:00,0
1,02.01.2022,00:00,00:00,0
2,03.01.2022,00:00,00:00,0
3,04.01.2022,00:00,00:00,0
4,05.01.2022,00:00,00:00,0


In [ ]:
def convert_str_to_float(text):
    if text == '-':
        return 0
    else:
        text=text.replace(',','.')
        return float(text)

# Test load all the companis by sectors 

In [331]:
DICT_COMPANIES_NAME_TO_CHANGE

{'Altenw_rth': 'Altenwoerth',
 'Heizkraftwerk_Altbach_Deizisau': 'Heizkraftwerk-Altbach/Deizisau',
 'GKH_St_cken': 'GKH-Stoecken',
 'H_usling': 'Haeusling',
 'S_ckingen': 'Saeckingen',
 'Ro_hag': 'Rosshag',
 'M_nchen_Nord_2': 'Muenchen-Nord-2',
 'Linz-S_d_Fernheizkraftwerk': 'Linz-Sued-Fernheizkraftwerk',
 'Gro_kraftwerk_Mannheim': 'Grosskraftwerk-Mannheim',
 'K_stenkraftwerk_K.I.E.L.': 'Kuestenkraftwerk-K.I.E.L',
 'Heizkraftwerk_R_merbr_cke': 'Heizkraftwerk-Roemerbruecke',
 'Heizkraftwerk_Dresden-Nossener_Br_cke': 'Heizkraftwerk-Dresden-Nossener-Bruecke',
 'Heizkraftwerk_M_nchen_S_d_GUD1_': 'Heizkraftwerk-Muenchen-Sued-GUD1',
 'Heizkraftwerk_M_nchen_S_d_GUD2': 'Heizkraftwerk-Muenchen-Sued-GUD2',
 'Heizkraftwerk_West__Frankfurt_': 'Heizkraftwerk-West-(Frankfurt)',
 'Kraftwerk_BASF_Ludwigshafen_S_d': 'Kraftwerk_BASF_Ludwigshafen_Sued'}

In [323]:
LIST_WEIRD_FIRST_RAW

['[MW] Berechnete Auflösungen',
 '(3 GT + 1 DT, Sammelschiene)',
 'GT 1, GT 2, DT 1 [MW] Berechnete Auflösungen']

In [332]:
dict_companies = load_company_data_by_sector()

Abwinden_Asten_202201010000_202212312359_Tag_1.csv
0 company =Abwinden_Asten.
2 company =Abwinden-Asten.
all columns= Index(['Datum', 'Anfang', 'Ende',
       'Generation_DE  [MW] Berechnete Auflösungen'],
      dtype='object')
nan_values False    99.178082
True      0.821918
Name: Generation_DE  [MW] Berechnete Auflösungen, dtype: float64
df.columns =  Index(['Datum', 'Anfang', 'Ende',
       'Generation_DE  [MW] Berechnete Auflösungen'],
      dtype='object')
company df that matches condition:
               Name      Sector
533  Abwinden-Asten  Laufwasser

*************************************************************************
Altenwoerth_202201010000_202212312359_Tag_3.csv
0 company =Altenwoerth.
2 company =Altenwoerth.
all columns= Index(['Datum', 'Anfang', 'Ende',
       'Generation_DE  [MW] Berechnete Auflösungen'],
      dtype='object')
nan_values False    98.90411
True      1.09589
Name: Generation_DE  [MW] Berechnete Auflösungen, dtype: float64
df.columns =  Index(['Datum',

/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:80: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace('.', '')
/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:80: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace('.', '')
/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:80: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[c

nan_values True    100.0
Name: Generation_DE 3.0 [MW] Berechnete Auflösungen, dtype: float64
nan_values False    99.178082
True      0.821918
Name: Generation_DE 5.0 [MW] Berechnete Auflösungen, dtype: float64
df.columns =  Index(['Datum', 'Anfang', 'Ende',
       'Generation_DE 4.0 [MW] Berechnete Auflösungen',
       'Generation_DE 5.0 [MW] Berechnete Auflösungen'],
      dtype='object')
company df that matches condition:
                      Name  Sector
215  Gaskraftwerk-Irsching  Erdgas

*************************************************************************
Gemeinschaftskernkraftwerk_Neckarwestheim_II_202201010000_202212312359_Tag_40.csv
0 company =Gemeinschaftskernkraftwerk_Neckarwestheim_II.
2 company =Gemeinschaftskernkraftwerk-Neckarwestheim-II.
all columns= Index(['Datum', 'Anfang', 'Ende',
       'Generation_DE GKN II [MW] Berechnete Auflösungen'],
      dtype='object')
nan_values False    99.178082
True      0.821918
Name: Generation_DE GKN II [MW] Berechnete Auflösunge

/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:80: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace('.', '')
/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:80: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace('.', '')
/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:80: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[c

nan_values False    98.90411
True      1.09589
Name: Generation_DE Süd GT2 [MW] Berechnete Auflösungen, dtype: float64
df.columns =  Index(['Datum', 'Anfang', 'Ende',
       'Generation_DE Süd GT3 [MW] Berechnete Auflösungen',
       'Generation_DE Süd GT2 [MW] Berechnete Auflösungen'],
      dtype='object')
company df that matches condition:
                                 Name  Sector
279  Heizkraftwerk-Muenchen-Sued-GUD1  Erdgas

*************************************************************************
Heizkraftwerk_M_nchen_S_d_GUD2_202201010000_202212312359_Tag_65.csv
0 company =Heizkraftwerk_M_nchen_S_d_GUD2.
1 company =Heizkraftwerk-Muenchen-Sued-GUD2.
2 company =Heizkraftwerk-Muenchen-Sued-GUD2.
all columns= Index(['Datum', 'Anfang', 'Ende',
       'Generation_DE Süd GT 61 [MW] Berechnete Auflösungen',
       'Generation_DE Süd DT 60 [MW] Berechnete Auflösungen',
       'Generation_DE Süd GT 62 [MW] Berechnete Auflösungen'],
      dtype='object')
nan_values False    98.90411
Tr

/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:80: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace('.', '')
/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:80: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace('.', '')
/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:80: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[c

0 company =Knapsack_Gas_I.
2 company =Knapsack-Gas-I.
all columns= Index(['Datum', 'Anfang', 'Ende',
       'Generation_DE I [MW] Berechnete Auflösungen'],
      dtype='object')
nan_values False    99.178082
True      0.821918
Name: Generation_DE I [MW] Berechnete Auflösungen, dtype: float64
df.columns =  Index(['Datum', 'Anfang', 'Ende',
       'Generation_DE I [MW] Berechnete Auflösungen'],
      dtype='object')
company df that matches condition:
               Name  Sector
779  Knapsack-Gas-I  Erdgas

*************************************************************************
Koepchenwerk_202201010000_202212312359_Tag_92.csv
0 company =Koepchenwerk.
2 company =Koepchenwerk.
all columns= Index(['Datum', 'Anfang', 'Ende',
       'Generation_DE Koepchenwerk [MW] Berechnete Auflösungen'],
      dtype='object')
nan_values False    98.90411
True      1.09589
Name: Generation_DE Koepchenwerk [MW] Berechnete Auflösungen, dtype: float64
df.columns =  Index(['Datum', 'Anfang', 'Ende',
       'G

/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:80: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace('.', '')
/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:80: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace('.', '')


IndexError: index 0 is out of bounds for axis 0 with size 0

In [189]:
comp = 'Heizkraftwerk_M_nchen_S_d_GUD2' 
if comp in DICT_COMPANIES_NAME_TO_CHANGE.keys():
    comp=DICT_COMPANIES_NAME_TO_CHANGE[comp]
    print(f'comp changed')
comp


'Heizkraftwerk_M_nchen_S_d_GUD2'